# Concept induction
This notebook offers sample [EDSL](https://docs.expectedparrot.com) code for using language models to identify concepts in unstructured texts, then generate criteria for the concepts, and then apply the criteria to evaluate the texts.

This idea is inspired by the recent paper: [Concept Induction: Analyzing Unstructured Text with High-Level Concepts Using LLooM](https://hci.stanford.edu/publications/2024/Lam_LLooM_CHI24.pdf)

## Technical setup
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Identify concepts
We start by creating a general question prompting the respondent (a language model) to identify concepts in a given text.

EDSL comes with a variety of [question types](https://docs.expectedparrot.com/en/latest/questions.html) that we can choose from based on the form of the response that we want to get back from the model. `QuestionList` may be appropriate where we want the response to be formatted as a list of strings:

In [1]:
from edsl import QuestionList

q_concepts = QuestionList(
    question_name="concepts",
    question_text="Identify the key concepts in the following text: {{ text }}",
    # max_list_items = # Optional
)

We might also want to ask some other questions about our data at the same time (a data labeling task). For example:

In [2]:
from edsl import QuestionMultipleChoice

q_sentiment = QuestionMultipleChoice(
    question_name="sentiment",
    question_text="Identify the sentiment of this text: {{ text }}",
    question_options=["Negative", "Neutral", "Positive"],
)

We parameterize the questions in order to run them for each of our texts. This is done with `Scenario` objects that we create for our data (here, some recent tweets by Pres. Biden):

In [3]:
# Replace with your data
texts = [  # POTUS recent tweets
    "Tune in as I deliver the keynote address at the U.S. Holocaust Memorial Museum’s Annual Days of Remembrance ceremony in Washington, D.C.",
    "We’re a nation of immigrants. A nation of dreamers. And as Cinco de Mayo represents, a nation of freedom.",
    "Medicare is stronger and Social Security remains strong. My economic plan has helped extend Medicare solvency by a decade. And I am committed to extending Social Security solvency by making the rich pay their fair share.",
    "Today, the Army Black Knights are taking home West Point’s 10th Commander-in-Chief Trophy. They should be proud. I’m proud of them too – not for the wins, but because after every game they hang up their uniforms and put on another: one representing the United States.",
    "This Holocaust Remembrance Day, we mourn the six million Jews who were killed by the Nazis during one of the darkest chapters in human history. And we recommit to heeding the lessons of the Shoah and realizing the responsibility of 'Never Again.'",
    "The recipients of the Presidential Medal of Freedom haven't just kept faith in freedom. They kept all of America's faith in a better tomorrow.",
    "Like Jill says, 'Teaching isn’t just a job. It’s a calling.' She knows that in her bones, and I know every educator who joined us at the White House for the first-ever Teacher State Dinner lives out that truth every day.",
    "Jill and I send warm wishes to Orthodox Christian communities around the world as they celebrate Easter. May the Lord bless and keep you this Easter Sunday and in the year ahead.",
    "Dreamers are our loved ones, nurses, teachers, and small business owners – they deserve the promise of health care just like all of us. Today, my Administration is making that real by expanding affordable health coverage through the Affordable Care Act to DACA recipients.",
    "With today’s report of 175,000 new jobs, the American comeback continues. Congressional Republicans are fighting to cut taxes for billionaires and let special interests rip folks off, I'm focused on job creation and building an economy that works for the families I grew up with.",
]
len(texts)

10

In [4]:
from edsl import ScenarioList

scenarios = ScenarioList.from_list("text", texts)
# scenarios

Next we combine the questions into a survey in order to administer them together (asynchronously by default, or according to any skip/stop rules or other logic that we want to add--learn more about `Survey` methods in our [documentation](https://docs.expectedparrot.com/en/latest/surveys.html)):

In [5]:
from edsl import Survey

survey = Survey(questions=[q_concepts, q_sentiment])

We add the scenarios to the survey and then run it to generate a dataset of results:

In [6]:
results = survey.by(scenarios).run()

EDSL comes with [built-in methods](https://docs.expectedparrot.com/en/latest/results.html) for working with results</a> in a variety of forms (data tables, SQL queries, dataframes, JSON, CSV). We can call the `columns` method to see a list of all the components that we can analyze:

In [7]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.concepts',
 'answer.sentiment',
 'comment.k_comment',
 'generated_tokens.concepts_generated_tokens',
 'generated_tokens.sentiment_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.concepts_system_prompt',
 'prompt.concepts_user_prompt',
 'prompt.sentiment_system_prompt',
 'prompt.sentiment_user_prompt',
 'question_options.concepts_question_options',
 'question_options.sentiment_question_options',
 'question_text.concepts_question_text',
 'question_text.sentiment_question_text',
 'question_type.concepts_question_type',
 'question_type.sentiment_question_type',
 'raw_model_response.concepts_cost',
 'raw_model_response.concepts_one_usd_buys',
 'raw_model_response.concepts_raw_model_response',
 'raw_model_response.sentiment_cost',
 'raw_model_response.sentiment_

We can select and print specific components to inspect in a table:

In [8]:
results.select("text", "concepts", "sentiment").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ scenario                                        ┃ answer                                           ┃ answer     ┃
┃ .text                                           ┃ .concepts                                        ┃ .sentiment ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ With today’s report of 175,000 new jobs, the    │ ['175,000 new jobs', 'American comeback',        │ Positive   │
│ American comeback continues. Congressional      │ 'Congressional Republicans', 'cut taxes for      │            │
│ Republicans are fighting to cut taxes for       │ billionaires', 'special interests', 'job         │            │
│ billionaires and let special interests rip      │ creation', 'economy that works for families']    │            │
│ folks off, I'm focused on job creation and      │                                                  │            │
│ building an economy that works for the families │                                                  │            │
│ I grew up with.                                 │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ Like Jill says, 'Teaching isn’t just a job.     │ ['Teaching is a calling', 'Jill', 'Teacher State │ Positive   │
│ It’s a calling.' She knows that in her bones,   │ Dinner', 'White House', 'educator']              │            │
│ and I know every educator who joined us at the  │                                                  │            │
│ White House for the first-ever Teacher State    │                                                  │            │
│ Dinner lives out that truth every day.          │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ Jill and I send warm wishes to Orthodox         │ ['Jill', 'Orthodox Christian communities',       │ Positive   │
│ Christian communities around the world as they  │ 'Easter', 'Lord bless', 'Easter Sunday', 'year   │            │
│ celebrate Easter. May the Lord bless and keep   │ ahead']                                          │            │
│ you this Easter Sunday and in the year ahead.   │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ Dreamers are our loved ones, nurses, teachers,  │ ['Dreamers', 'health care', 'Affordable Care     │ Positive   │
│ and small business owners – they deserve the    │ Act', 'DACA recipients']                         │            │
│ promise of health care just like all of us.     │                                                  │            │
│ Today, my Administration is making that real by │                                                  │            │
│ expanding affordable health coverage through    │                                                  │            │
│ the Affordable Care Act to DACA recipients.     │                                                  │            │
├─────────────────────────────────────────────────┼──────────────────────────────────────────────────┼────────────┤
│ The recipients of the Presidential Medal of     │ ['Presidential Medal of Freedom', 'faith in      │ Positive   │
│ Freedom haven't just kept faith in freedom.     │ freedom', "America's faith", 'better tomorrow']  │            │
│ They kept all of America's faith in a better    │                                                  │            │
│ tomorrow.                                       │                                                  │            │
├─────────────────────────────────────────────────┼─────

If our concepts lists are too long, we can run another question prompting a model to condense it. We can specify the number of concepts that we want to get:

In [9]:
# Flattening our list of lists for all the texts to use in a follow-on question:
concepts_list = results.select("concepts").to_list(flatten=True)
# concepts_list

In [10]:
q_condense = QuestionList(
    question_name="condense",
    question_text="Return a condensed list of the following list of concepts: "
    + ", ".join(concepts_list),
    max_list_items=10,
)

Note that we can call the `run()` method on either a survey of questions or an individual question:

In [11]:
results = q_condense.run()

In [12]:
results.select("condense").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ answer                                                                                                          ┃
┃ .condense                                                                                                       ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ ['Job creation', 'Economy for families', 'Teaching as a calling', 'Orthodox Christian Easter', 'Affordable Care │
│ Act', 'DACA recipients', 'Holocaust Remembrance', 'Medicare and Social Security solvency', 'Nation of           │
│ immigrants', 'U.S. Holocaust Memorial Museum']                                                                  │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

## Identify criteria for each concept
Similar to our first step, next we can run a question prompting the model to generate criteria for each concept. We could use `QuestionFreeText` to generate criteria in an unstructured narrative:

In [13]:
from edsl import QuestionFreeText

q_criteria = QuestionFreeText(
    question_name="criteria",
    question_text="""Describe key criteria for determining whether a text is primarily about the 
    following concept: {{ concept }}""",
)

For this question, the scenarios are the concepts that we generated:

In [14]:
condensed_concepts_list = results.select("condense").to_list(flatten=True)

scenarios = ScenarioList.from_list("concept", condensed_concepts_list)
scenarios

ScenarioList([Scenario({'concept': 'Job creation'}), Scenario({'concept': 'Economy for families'}), Scenario({'concept': 'Teaching as a calling'}), Scenario({'concept': 'Orthodox Christian Easter'}), Scenario({'concept': 'Affordable Care Act'}), Scenario({'concept': 'DACA recipients'}), Scenario({'concept': 'Holocaust Remembrance'}), Scenario({'concept': 'Medicare and Social Security solvency'}), Scenario({'concept': 'Nation of immigrants'}), Scenario({'concept': 'U.S. Holocaust Memorial Museum'})])

In [15]:
results = q_criteria.by(scenarios).run()

In [16]:
results.select("concept", "criteria").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                              ┃ answer                                                                  ┃
┃ .concept                              ┃ .criteria                                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Job creation                          │ To determine whether a text is primarily about the concept of job       │
│                                       │ creation, you can look for several key criteria:                        │
│                                       │                                                                         │
│                                       │ 1. **Main Topic Focus**: The text should have job creation as its       │
│                                       │ central theme. This means the bulk of the content discusses how jobs    │
│                                       │ are being created, the factors influencing job creation, or the         │
│                                       │ outcomes of job creation efforts.                                       │
│                                       │                                                                         │
│                                       │ 2. **Economic Indicators**: The text should reference economic          │
│                                       │ indicators related to job creation, such as employment rates,           │
│                                       │ unemployment rates, new business startups, and job growth statistics.   │
│                                       │                                                                         │
│                                       │ 3. **Policy and Programs**: Look for discussions about government       │
│                                       │ policies, initiatives, or programs designed to stimulate job creation.  │
│                                       │ This could include tax incentives for businesses, workforce development │
│                                       │ programs, or infrastructure projects.                                   │
│                                       │                                                                         │
│                                       │ 4. **Case Studies and Examples**: The text should provide examples or   │
│                                       │ case studies of successful job creation efforts. This could involve     │
│                                       │ specific industries, regions, or companies that have created a          │
│                                       │ significant number of jobs.                                             │
│                                       │                                                                         │
│                                       │ 5. **Challenges and Solutions**: The text might explore challenges to   │
│                                       │ job creation, such as economic downturns, automation, or regulatory     │
│                                       │ hurdles, and propose solutions to these challenges.                     │
│                                       │                                                                         │
│                                       │ 6. **Stakeholder Involvement**: The text should mention key             │
│                                       │ stakeholders involved in job creation, including government entities,   │
│                                       │ private sector companies, non-profits, and educational institutions.    │
│                                       │                                                                         │
│                                       │ 7. **Future Pr

## Identify the concepts in each text and evaluate based on the criteria
Finally, we can use the concepts and the criteria to run another question where we prompt the model to evaulate each text. Question types `QuestionLinearScale`, `QuestionRank` or `QuestionNumerical` may be appropriate where we want to return a score:

In [17]:
from edsl import QuestionLinearScale

q_score = QuestionLinearScale(
    question_name="score",
    question_text="""Consider the following concept and criteria for determining whether 
    a given text addresses this concept. Then score how well the following text satisfies
    the criteria for the concept.
    Concept: {{ concept }}
    Criteria: {{ criteria }}
    Text: {{ text }}""",
    question_options=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    option_labels={0: "Not at all", 10: "Very well"},  # Optional
)

Here we want to use both the texts and the concepts and corresponding criteria together as scenarios of the question:

In [18]:
concepts_criteria = [
    list(pair)
    for pair in zip(
        results.select("concept").to_list(), results.select("criteria").to_list()
    )
]
len(concepts_criteria)

10

In [19]:
from edsl import ScenarioList, Scenario

scenarios = ScenarioList(
    Scenario({"text": text, "concept": concept, "criteria": criteria})
    for text in texts
    for [concept, criteria] in concepts_criteria
)

In [20]:
results = q_score.by(scenarios).run()

We can filter the results based on the responses--e.g., here we just show the non-zero scores:

In [21]:
(
    results.filter("score > 0")
    .select("text", "concept", "score")
    .print(format="rich")
)

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ scenario                                                       ┃ scenario                              ┃ answer ┃
┃ .text                                                          ┃ .concept                              ┃ .score ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ Dreamers are our loved ones, nurses, teachers, and small       │ Economy for families                  │ 2      │
│ business owners – they deserve the promise of health care just │                                       │        │
│ like all of us. Today, my Administration is making that real   │                                       │        │
│ by expanding affordable health coverage through the Affordable │                                       │        │
│ Care Act to DACA recipients.                                   │                                       │        │
├────────────────────────────────────────────────────────────────┼───────────────────────────────────────┼────────┤
│ With today’s report of 175,000 new jobs, the American comeback │ Economy for families                  │ 3      │
│ continues. Congressional Republicans are fighting to cut taxes │                                       │        │
│ for billionaires and let special interests rip folks off, I'm  │                                       │        │
│ focused on job creation and building an economy that works for │                                       │        │
│ the families I grew up with.                                   │                                       │        │
├────────────────────────────────────────────────────────────────┼───────────────────────────────────────┼────────┤
│ With today’s report of 175,000 new jobs, the American comeback │ Job creation                          │ 2      │
│ continues. Congressional Republicans are fighting to cut taxes │                                       │        │
│ for billionaires and let special interests rip folks off, I'm  │                                       │        │
│ focused on job creation and building an economy that works for │                                       │        │
│ the families I grew up with.                                   │                                       │        │
├────────────────────────────────────────────────────────────────┼───────────────────────────────────────┼────────┤
│ Medicare is stronger and Social Security remains strong. My    │ Economy for families                  │ 3      │
│ economic plan has helped extend Medicare solvency by a decade. │                                       │        │
│ And I am committed to extending Social Security solvency by    │                                       │        │
│ making the rich pay their fair share.                          │                                       │        │
├────────────────────────────────────────────────────────────────┼───────────────────────────────────────┼────────┤
│ Dreamers are our loved ones, nurses, teachers, and small       │ Affordable Care Act                   │ 6      │
│ business owners – they deserve the promise of health care just │                                       │        │
│ like all of us. Today, my Administration is making that real   │                                       │        │
│ by expanding affordable health coverage through the Affordable │                                       │        │
│ Care Act to DACA recipients.                                   │                                       │        │
├────────────────────────────────────────────────────────────────┼───────────────────────────────────────┼────────┤
│ Dreamers are our loved ones, nurses, teachers, and small       │ DACA recipients                       │ 5      │
│ business owners – they deserve the promise of health c

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/content/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the scenarios, survey and results from above, and this notebook:

In [22]:
scenarios.push(description = "Example scenarios", visibility = "public")

{'description': 'Example scenarios',
 'object_type': 'scenario_list',
 'url': 'https://www.expectedparrot.com/content/5c1f6856-32e4-4473-97e6-928541759637',
 'uuid': '5c1f6856-32e4-4473-97e6-928541759637',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

In [23]:
survey.push(description = "Example survey", visibility = "public")

{'description': 'Example survey',
 'object_type': 'survey',
 'url': 'https://www.expectedparrot.com/content/2cf3c5fd-e6c1-4135-af96-0ce866dc28bb',
 'uuid': '2cf3c5fd-e6c1-4135-af96-0ce866dc28bb',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

In [24]:
results.push(description = "Example results", visibility = "public")

{'description': 'Example results',
 'object_type': 'results',
 'url': 'https://www.expectedparrot.com/content/4d7b3230-575e-47d0-b321-81b59d2df16f',
 'uuid': '4d7b3230-575e-47d0-b321-81b59d2df16f',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

We can also post this notebook:

In [25]:
from edsl import Notebook

In [26]:
n = Notebook(path = "concept_induction.ipynb")

In [27]:
n.push(description = "Example code for concept induction", visibility = "public")

{'description': 'Example code for concept induction',
 'object_type': 'notebook',
 'url': 'https://www.expectedparrot.com/content/6f29a7b3-6a2e-460b-bf39-baeb7d6c39a1',
 'uuid': '6f29a7b3-6a2e-460b-bf39-baeb7d6c39a1',
 'version': '0.1.33.dev1',
 'visibility': 'public'}

To update an object at the Coop:

In [28]:
n = Notebook(path = "concept_induction.ipynb") # resave it

In [29]:
n.patch(uuid = "6f29a7b3-6a2e-460b-bf39-baeb7d6c39a1", value = n)

{'status': 'success'}